<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/aumentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aumentacion de datos
### Version 1.0
## Fotos de entrada en: Detectadas
## Fotos de salida en: Aumentadas

In [ ]:
# prompt: abrir google drive

from google.colab import drive

# Unmount the drive first if it is mounted
try:
  drive.flush_and_unmount()
except ValueError:
  pass

# Mount the drive to the desired path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# List files in your Drive
!ls /content/drive/MyDrive


Mounted at /content/drive
 BackPropagation.ipynb	 Detectadas   'Hoja de cálculo sin título.gsheet'
 caras			 Detectadas2   labo1
'Colab Notebooks'	 DMA	       shape_predictor_68_face_landmarks.dat
 datos_isomap.pkl	 fotos	       viejos


In [ ]:
import os
from PIL import Image, ImageOps
from pathlib import Path



In [ ]:
# Carpeta de origen y destino
carpeta_origen = Path("/content/drive/MyDrive/Detectadas")
carpeta_destino = Path("/content/drive/MyDrive/Aumentadas")


In [ ]:
# Crear carpeta destino si no existe
carpeta_destino.mkdir(parents=True, exist_ok=True)

# Transformaciones
desplazamientos = [-10, 10]  # en px
rotaciones = [-10, 10]  # en grados

def desplazar(img, dx=0, dy=0):
    """Desplaza una imagen dentro del lienzo original."""
    fondo = Image.new("L", img.size, color=0)  # Asumimos imágenes en escala de grises
    fondo.paste(img, (dx, dy))
    return fondo

def guardar_variacion(imagen, nombre_base, sufijo, carpeta):
    """Guarda la imagen con sufijo en la carpeta destino."""
    nombre_archivo = f"{nombre_base}_{sufijo}.png"
    ruta_guardado = carpeta / nombre_archivo
    imagen.save(ruta_guardado)


In [ ]:
# Recorrer subcarpetas e imágenes
for subcarpeta in carpeta_origen.iterdir():
    if subcarpeta.is_dir():
        sub_destino = carpeta_destino / subcarpeta.name
        sub_destino.mkdir(parents=True, exist_ok=True)

        for archivo in subcarpeta.glob("*.jp*g"):  # Suponiendo imágenes .jpg
            imagen = Image.open(archivo).convert("L") # Ensure grayscale for consistent displacement background
            nombre_base = archivo.stem

            # Get image dimensions for percentage calculations
            ancho, alto = imagen.size
            desplazamiento_horizontal = int(ancho * 0.05)
            desplazamiento_vertical = int(alto * 0.05)

            # Guardar imagen original
            #imagen.save(sub_destino / f"{nombre_base}_original.jpg")

            # --- Apply transformations based on the original image ---

            # a) Desplazamientos horizontales (5% izquierda y derecha)
            img_dx_left = desplazar(imagen, dx=-desplazamiento_horizontal)
            guardar_variacion(img_dx_left, nombre_base, f"dx{-int(desplazamiento_horizontal/ancho*100)}pct", sub_destino)

            img_dx_right = desplazar(imagen, dx=desplazamiento_horizontal)
            guardar_variacion(img_dx_right, nombre_base, f"dx{int(desplazamiento_horizontal/ancho*100)}pct", sub_destino)

            # b) Desplazamientos verticales (5% arriba y abajo)
            img_dy_up = desplazar(imagen, dy=-desplazamiento_vertical)
            guardar_variacion(img_dy_up, nombre_base, f"dy{-int(desplazamiento_vertical/alto*100)}pct", sub_destino)

            img_dy_down = desplazar(imagen, dy=desplazamiento_vertical)
            guardar_variacion(img_dy_down, nombre_base, f"dy{int(desplazamiento_vertical/alto*100)}pct", sub_destino)

            # c) Rotaciones (10 grados horario y antihorario)
            img_rot_neg10 = imagen.rotate(-10, expand=True, fillcolor=0)
            guardar_variacion(img_rot_neg10, nombre_base, "rot-10", sub_destino)

            img_rot_pos10 = imagen.rotate(10, expand=True, fillcolor=0)
            guardar_variacion(img_rot_pos10, nombre_base, "rot10", sub_destino)

            # Note: We removed the horizontal flip and the subsequent rotations
            # on variations as per your request to only use the original image.